In [1]:
import os
!apt-get update -qq > /dev/null   
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
!pip install nlu  pyspark==2.4.7 > /dev/null   
!pip install flask_ngrok
import nlu

In [2]:
API = "http://ec2-3-108-194-64.ap-south-1.compute.amazonaws.com:5000/api/"

In [ ]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok
import json
import math
import requests

app = Flask(__name__)
run_with_ngrok(app)

multi_pipe = None
col_names = ['sentence_embedding_bert','sentence_embedding_electra', 'sentence_embedding_use']

@app.route("/load-api")
def loadAPI():
    try:
        global multi_pipe
        multi_pipe = nlu.load('use en.embed_sentence.electra embed_sentence.bert')
        return {"status" : 200},200
    except:
        return {"status" : 500},500

@app.route("/generate-predictions",methods = ['GET', 'POST'])
def genearatePredictions():

    title = request.form['title']
    
    if title and multi_pipe:
        predictions = multi_pipe.predict(title,get_embeddings=True).iloc[0]
        
        response = {}

        for col_name in col_names:
            response[col_name] = list(predictions[col_name][0])
        
        return { "predictions" : response ,"status" : 201},201
    else:
        return { "error" :"ERROR" ,"status" : 500}, 500

def dot(A,B): 
    return (sum(a*b for a,b in zip(A,B)))

def cosine_similarity(a,b):
    return dot(a,b) / ( math.sqrt(dot(a,a) * dot(b,b)) )

@app.route("/get-similar-questions",methods=['GET','POST'])
def getSimilarQuestions():
    data = requests.get(API+"discussion/predictions").json()
    embeddings = eval(request.form['predictions'])
    print(embeddings)
    if data['status'] == 200 and embeddings:
        predictions = data['predictions']
        for i in range(len(predictions)):
            similarity = 0
            for e_col in col_names:
                if e_col in predictions[i]['predictions'] and e_col in embeddings:
                    similarity += cosine_similarity(predictions[i]['predictions'][e_col],embeddings[e_col])

            predictions[i]['similarity_score'] = similarity/len(col_names)
        
        return {"similarQuestions" : sorted(predictions,key = lambda x : x['similarity_score'],reverse=True)[:3] , "status" : 201} , 201
    else:
        return { "status" : 500,"error" :"ERROR"}, 500

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d0f1-34-134-248-125.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sent_electra_small_uncased download started this may take some time.
Approximate size to download 48.7 MB
[OK!]
sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]

127.0.0.1 - - [29/Nov/2021 11:47:03] "GET /load-api HTTP/1.1" 200 -


127.0.0.1 - - [29/Nov/2021 11:47:25] "POST /generate-predictions HTTP/1.1" 201 -
127.0.0.1 - - [29/Nov/2021 11:47:55] "POST /get-similar-questions HTTP/1.1" 201 -


...

{'sentence_embedding_bert': [-1.0026072263717651, -0.11482470482587814, 0.14700667560100555, -1.3783165216445923, 0.9083464741706848, -0.11403165757656097, -0.24702051281929016, 0.6442373394966125, -1.3414437770843506, -0.3408529460430145, 0.7336907982826233, 0.4534391760826111, -0.06295717507600784, 0.6339627504348755, 1.6095539331436157, -1.3990644216537476, -0.9619437456130981, -0.0766419842839241, -1.213323950767517, 1.1027065515518188, -0.6720837354660034, -0.012245957739651203, 1.5394361019134521, 0.7732383012771606, 1.299834132194519, 0.25084325671195984, -0.08592887222766876, 0.701392650604248, 1.125167727470398, -1.2786548137664795, -0.7262722253799438, -2.5498087406158447, -0.5865354537963867, -0.9266512989997864, 0.5531934499740601, -0.6053912043571472, -1.0591018199920654, -0.5920910239219666, -3.2492010593414307, -1.4748984575271606, 0.9197390675544739, -0.42846688628196716, 0.4651246964931488, -1.1563160419464111, 0.9724583029747009, -0.8528262972831726, -0.69530624151229

127.0.0.1 - - [29/Nov/2021 11:54:55] "POST /generate-predictions HTTP/1.1" 201 -
127.0.0.1 - - [29/Nov/2021 11:54:57] "POST /get-similar-questions HTTP/1.1" 201 -


{'sentence_embedding_bert': [-1.5094188451766968, 0.32611680030822754, 0.1244254931807518, -1.3967442512512207, 0.5039196610450745, 0.1974066197872162, -0.9194619059562683, 0.8473272323608398, -1.6643308401107788, -0.7945162057876587, 0.45514997839927673, 0.6392819881439209, 0.3794522285461426, 0.5420401692390442, 1.849405288696289, -1.0904735326766968, -1.372143030166626, 0.28596386313438416, -0.623656153678894, 0.7865938544273376, -0.5326036810874939, -0.36605262756347656, 1.3852896690368652, 0.6085406541824341, 1.4746384620666504, -0.08358363062143326, 0.4663692116737366, 1.7922197580337524, 0.014363742433488369, -0.686901330947876, -0.3213561177253723, -2.303088903427124, -1.4065873622894287, -0.286615252494812, 0.42649781703948975, 0.8765962719917297, 0.21035516262054443, -0.007225910201668739, -1.0168265104293823, -1.0061390399932861, 0.49398815631866455, -0.2404652237892151, 0.22191359102725983, -0.7110984325408936, -0.3283602297306061, -1.0920158624649048, -0.9187753200531006, 

127.0.0.1 - - [29/Nov/2021 11:57:48] "POST /generate-predictions HTTP/1.1" 201 -
127.0.0.1 - - [29/Nov/2021 11:57:50] "POST /get-similar-questions HTTP/1.1" 201 -


{'sentence_embedding_bert': [-1.5094188451766968, 0.32611680030822754, 0.1244254931807518, -1.3967442512512207, 0.5039196610450745, 0.1974066197872162, -0.9194619059562683, 0.8473272323608398, -1.6643308401107788, -0.7945162057876587, 0.45514997839927673, 0.6392819881439209, 0.3794522285461426, 0.5420401692390442, 1.849405288696289, -1.0904735326766968, -1.372143030166626, 0.28596386313438416, -0.623656153678894, 0.7865938544273376, -0.5326036810874939, -0.36605262756347656, 1.3852896690368652, 0.6085406541824341, 1.4746384620666504, -0.08358363062143326, 0.4663692116737366, 1.7922197580337524, 0.014363742433488369, -0.686901330947876, -0.3213561177253723, -2.303088903427124, -1.4065873622894287, -0.286615252494812, 0.42649781703948975, 0.8765962719917297, 0.21035516262054443, -0.007225910201668739, -1.0168265104293823, -1.0061390399932861, 0.49398815631866455, -0.2404652237892151, 0.22191359102725983, -0.7110984325408936, -0.3283602297306061, -1.0920158624649048, -0.9187753200531006, 